In [5]:
#pip install langchain-chroma
#!pip install pypdf

In [6]:
# import
import warnings
warnings.filterwarnings("ignore")
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
# load the document and split it into chunks
loader = PyPDFLoader("D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf")
documents = loader.load()

In [8]:
documents

[Document(page_content='Indian Economy: An Overview\nIndian Economy: An Overview\nPage 1', metadata={'source': 'D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf', 'page': 0}),
 Document(page_content="Indian Economy: An Overview\nIntroduction\nIndia, one of the world's fastest-growing economies, has shown remarkable resilience and growth\nover the past few decades. With a population of over 1.4 billion people, it is the world's second-most\npopulous country and boasts a diverse and complex economy that encompasses agriculture,\nmanufacturing, and services sectors. This document provides an overview of the Indian economy,\ntracing its historical background, economic reforms, key sectors, current scenario, challenges, and\nfuture prospects.\nPage 2", metadata={'source': 'D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf', 'page': 1}),
 Document(page_content="Indian Economy: An Overview\nHistorical Background\nThe economic history of India is a story of contrast and transf

In [56]:
# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
docs = text_splitter.split_documents(documents)
docs

[Document(page_content='Indian Economy: An Overview\nIndian Economy: An Overview\nPage 1', metadata={'source': 'D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf', 'page': 0}),
 Document(page_content="Indian Economy: An Overview\nIntroduction\nIndia, one of the world's fastest-growing economies, has shown remarkable resilience and growth\nover the past few decades. With a population of over 1.4 billion people, it is the world's second-most\npopulous country and boasts a diverse and complex economy that encompasses agriculture,\nmanufacturing, and services sectors. This document provides an overview of the Indian economy,", metadata={'source': 'D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf', 'page': 1}),
 Document(page_content='tracing its historical background, economic reforms, key sectors, current scenario, challenges, and\nfuture prospects.\nPage 2', metadata={'source': 'D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf', 'page': 1}),
 Document(page_conte

In [57]:
len(docs)

17

In [58]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [39]:
text = "This is a test document."

In [59]:
query_result = embeddings.embed_query(text)
len(query_result)

384

In [19]:
#pip install langchain-chroma

In [20]:
from langchain_chroma import Chroma

In [60]:
db = Chroma.from_documents(docs, embeddings)

In [24]:
from langchain.vectorstores import chroma
db1 = Chroma.from_documents(docs, embeddings)


In [62]:
query = "Future Prospects of Indian Economy?"
matching_docs = db.similarity_search(query)
matching_docs[0]

Document(page_content='Indian Economy: An Overview\nIndian Economy: An Overview\nPage 1', metadata={'page': 0, 'source': 'D:\\Data\\LLM\\IndianEconomy\\Indian_Economy_Overview.pdf'})

In [30]:
persist_directory = "D:\\CromaDB"
vectordb = Chroma.from_documents(
    documents=docs, embedding=embeddings, persist_directory=persist_directory
)

# # save to disk
# db2 = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
# docs = db2.similarity_search(query)

In [63]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

In [64]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

#query = "Key Sectors of the Indian Economy?"
matching_docs = db.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Question: Future Prospects of Indian Economy?
Helpful Answer:

> Finished chain.

> Finished chain.


"I don't know."

In [65]:
query = "Key Sectors of the Indian Economy?"
db3 = Chroma(persist_directory="D:\\CromaDB", embedding_function=embeddings)
docs = db3.similarity_search(query)
print(docs[0].page_content)

Indian Economy: An Overview
Indian Economy: An Overview
Page 1


In [66]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff",verbose=True)

query = "Key Sectors of the Indian Economy?"
matching_docs = db3.similarity_search(query)
answer =  chain.run(input_documents=matching_docs, question=query)
answer



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Indian Economy: An Overview
Indian Economy: An Overview
Page 1

Indian Economy: An Overview
Introduction
India, one of the world's fastest-growing economies, has shown remarkable resilience and growth
over the past few decades. With a population of over 1.4 billion people, it is the world's second-most
populous country and boasts a diverse and complex economy that encompasses agriculture,
manufacturing, and services sectors. This document provides an overview of the Indian economy,
tracing its historical background, economic reforms, key sectors, current scenario, challenges, and
future prospect

'According to the provided context, the helpful answer is:\n\nAgriculture, manufacturing, and services sectors.\n\nThese three sectors are mentioned as the areas that the Indian economy encompasses.'